## Web scrapping using python

#### References
1. [Practical Introduction to Web Scraping in Python](https://realpython.com/python-web-scraping-practical-introduction/)
2. [Web Scraping using Python](https://www.datacamp.com/community/tutorials/web-scraping-using-python)

In [ ]:
# $ python3 -m venv venv
# $ . ./venv/bin/activate

In [1]:
#Better
!pip install requests BeautifulSoup4 fire

  Created wheel for fire: filename=fire-0.3.1-py2.py3-none-any.whl size=111010 sha256=aedbdaf45bf5c73f48c13fc579c2cc6eba14f396b50f1d5f190c615190a7a328
  Stored in directory: c:\users\jeannette\appdata\local\pip\cache\wheels\95\38\e1\8b62337a8ecf5728bdc1017e828f253f7a9cf25db999861bec
  Created wheel for termcolor: filename=termcolor-1.1.0-py3-none-any.whl size=4835 sha256=680b661b797c9bea096d3b40b5accdf6ca43f6b44fe6b8370edbee61f91272b9
  Stored in directory: c:\users\jeannette\appdata\local\pip\cache\wheels\3f\e3\ec\8a8336ff196023622fbcb36de0c5a5c218cbb24111d1d4c7f2
Successfully built fire termcolor


In [10]:
from requests import get
from requests.exceptions import RequestException
from contextlib import closing
from bs4 import BeautifulSoup
import pandas as pd
import os, sys
import matplotlib.pyplot as plt
import seaborn as sns
import tweepy
from tweepy import OAuthHandler
from tweepy import API
from tweepy import Cursor
from datetime import datetime, date, time, timedelta
from collections import Counter
import sys

%matplotlib inline

import fire

In [11]:
#%%writefile ../pyscrap_url.py

def simple_get(url):
    """
    Attempts to get the content at `url` by making an HTTP GET request.
    If the content-type of response is some kind of HTML/XML, return the
    text content, otherwise return None.
    """
    try:
        with closing(get(url, stream=True)) as resp:
            if is_good_response(resp):
                return resp.content  #.encode(BeautifulSoup.original_encoding)
            else:
                return None

    except RequestException as e:
        log_error('Error during requests to {0} : {1}'.format(url, str(e)))
        return None


def is_good_response(resp):
    """
    Returns True if the response seems to be HTML, False otherwise.
    """
    content_type = resp.headers['Content-Type'].lower()
    return (resp.status_code == 200 
            and content_type is not None 
            and content_type.find('html') > -1)


def log_error(e):
    """
    It is always a good idea to log errors. 
    This function just prints them, but you can
    make it do anything.
    """
    print(e)
    
def get_elements(url, tag='',search={}, fname=None):
    """
    Downloads a page specified by the url parameter
    and returns a list of strings, one per tag element
    """
    if isinstance(url,str):
        response = simple_get(url)
    else:
        #if already it is a loaded html page
        response = url
    if response is not None:
        html = BeautifulSoup(response, 'html.parser')
        res = []
        if tag:    
            for li in html.select(tag):
                for name in li.text.split('\n'):
                    if len(name) > 0:
                        res.append(name.strip())
        if search:
            soup = html            
            r = ''
            if 'find' in search.keys():
                print('findaing',search['find'])
                soup = soup.find(**search['find'])
                r = soup
            if 'find_all' in search.keys():
                print('findaing all of',search['find_all'])
                r = soup.find_all(**search['find_all'])
            if r:
                for x in list(r):
                    if len(x) > 0:
                        res.extend(x)
        return res
    # Raise an exception if we failed to get any data from the url
    raise Exception('Error retrieving contents at {}'.format(url))    
    

    
if get_ipython().__class__.__name__ == '__main__':
    fire(get_elements)

In [56]:
res = get_elements('https://africafreak.com/100-most-influential-twitter-users-in-africa', tag = 'h2')
df = pd.DataFrame(res)
df.head(10)

,0
0,100. Jeffrey Gettleman (@gettleman)
1,99. Africa24 Media (@a24media)
2,98. Scapegoat (@andiMakinana)
3,97. Africa Check (@AfricaCheck)
4,96. James Copnall (@JamesCopnall)
5,95. Online Africa (@oafrica)
6,94. Patrick Ngowi (@PatrickNgowi)
7,93. DOS African Affairs (@StateAfrica)
8,92. MoadowAJE (@Moadow)
9,91. Brendan Boyle (@BrendanSAfrica)


In [17]:
url= 'https://www.atlanticcouncil.org/blogs/africasource/african-leaders-respond-to-coronavirus-on-twitter/#east-africa'
response = simple_get(url)


In [54]:
res_gov = get_elements(response, tag='blockquote')
for r in res_gov:
    split_data = r.split('— ',maxsplit=1)[1].rsplit('(',maxsplit=1)
    name = split_data[0].split(',')[0].strip()
    handle =  split_data[1].rsplit(')',maxsplit=1)[0]
    print(f'{name} : {handle}')


Eswatini Government : @EswatiniGovern1
Malawi Government : @MalawiGovt
Hage G. Geingob : @hagegeingob
Seychelles Ministry of Finance : @FinanceSC
PresidencyZA : @PresidencyZA
Ministry of Health Zambia : @mohzambia
President of Zimbabwe : @edmnangagwa
MinSantédj : @MinSantedj
Yemane G. Meskel : @hawelti
State House Kenya : @StateHouseKenya
Paul Kagame : @PaulKagame
Mohamed Farmaajo : @M_Farmaajo
H.E Hussein Abdelbagi Akol : @SouthSudanGov
Abdalla Hamdok : @SudanPMHamdok
TanzaniaSpokesperson : @TZSpokesperson
Yoweri K Museveni : @KagutaMuseveni
MOFA/MRE -(Angola) : @angola_Mirex
Amb. Willy Nyamitwe : @willynyamitwe
Chérif Mahamat Zene : @Cherif_MZ
Présidence RDC 🇨🇩 : @Presidence_RDC
Ali Bongo Ondimba : @PresidentABO
Présidence du Bénin : @PresidenceBenin
Roch KABORE : @rochkaborepf
Presidente Cabo Verde : @PresidenciaCV
Alassane Ouattara : @AOuattara_PRCI
State House of The Gambia : @Presidency_GMB
Nana Akufo-Addo : @NAkufoAddo
Pr. Alpha CONDÉ : @President_GN
Umaro Sissoco Embalo : @USEm

## Web scrapping using bash script
If the web site has a quite simple HTML, you can easily use curl to perform the request and then extract the needed values using bash commands grep, cut , sed, ..

This tutorial is adapted from [this](https://medium.com/@LiliSousa/web-scraping-with-bash-690e4ee7f98d) medium article

In [59]:
#twitter influencers data cleaning
df1 = df[0].str.split('.',expand = True)
df1.head(10)
df2 = df1[1].str.split('(',expand = True)
df2.head(10)
df3 = df2[1].str.split(')',expand = True)
df3.tail(20)
df4 = df3.dropna(axis=0, how='any')
df5 = df4[0].replace('@','', regex=True)
df5.tail(10).to_csv (r'C:\Users\jeannette\Desktop\export_dataframe.csv', index = False, header=True)
df5.tail(10)


89      MTVbaseAfrica
90        Computicket
91         loyisogola
92                5FM
93    mailandguardian
94         helenzille
95    Julius_S_Malema
96             News24
98        GarethCliff
99         Trevornoah
Name: 0, dtype: object

In [12]:
consumer_key = '6DiGOBXi1BqSNYQkXNLxcQUPY'
consumer_secret = 'kQHjfN7LEZQjHCgD447qAS95zpWN8xajCgf8W6pAt1RIPzMqBZ'
access_token = '855442991695224833-nDKgA31lMS1CKylDAJlOe4kEaVzidwg'
access_token_secret = 'oxDeKQ55OVx5TGdXnrDUGByKxqoGLTUBI8ygqt8AZqiDk'
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = API(auth)

In [55]:
for target in df5:
    try:
        
        print("Getting data for " + target)
        tweets = api.user_timeline(screen_name=target,count=5)
        tweet_list = [tweet for tweet in tweets]
        counter = 0
        favcounter= 0
        for tweet in tweet_list:
            retweetcount = tweet.retweet_count
            favorite = tweet.favorite_count
            counter += retweetcount
            favcounter += favorite
            user_details = api.get_user(target)
            followers_count = user_details.followers_count
            friends_count = user_details.friends_count
            print(tweet.text)
            print('number of retweets ' , retweetcount)
            print('number of likes ' , favorite)
            popularity_score = counter + favcounter  
            reach_score = followers_count - friends_count
        print('POPULARITY SCORE',popularity_score)
        print('Reach score', reach_score )
    except tweepy.TweepError :
        pass

Getting data for gettleman
RT @SavvyPriya: @gettleman @HariNYT It's a very charming article on India's response to coronavirus. How Indian culture has defined the res…
number of retweets  1
number of likes  0
RT @KetanDayal: How world's 2 biggest troops operates at 15,000 ft. A ground report based on inputs from Ladakhis, retd. officials of ITBP&amp;…
number of retweets  1
number of likes  0
The illustrious Bollywood Bachchan clan tests positive for Covid. How did they get it? https://t.co/zvtAjJPIrh
number of retweets  7
number of likes  26
Dan Eldon was killed on this date in 1992. He squeezed the hell out of every day. I was lucky to know him.… https://t.co/Lyf9pJCrG9
number of retweets  4
number of likes  19
RT @Chun_Gurung: “He’s like god,” said Shivendra Singh Dungarpur, a filmmaker who has worked with @SrBachchan. “I’ve never seen a star havi…
number of retweets  1
number of likes  0
POPULARITY SCORE 59
Reach score 25665
Getting data for a24media
RT @Yebo_Live: Saro-Wiwa &amp; 8

Brendan Boyle, my father, died peacefully at home on Sunday evening. The tragedy of his death cannot be overstated,… https://t.co/YWkEPg7ic3
number of retweets  12
number of likes  196
RT @ChrisRoper: Trump’s govt shutdown, Zille’s tax revolt. Grand and empty political gestures that screw over the people you profess to car…
number of retweets  6
number of likes  0
POPULARITY SCORE 329
Reach score 28095
Getting data for CityTshwane
Stage 2 #Loadshedding for 15 July 2020 from 14h00

1300 to 15h30: Group 2 and  6

15h00 to 17h30: Group 3 and 7

17… https://t.co/jPAPvnTdQC
number of retweets  28
number of likes  54
How to report power interruptions. https://t.co/OvQaXxZhCa
number of retweets  4
number of likes  8
@Mbongeni_Owen Good day, please report power interruption by SMSing the word 'power' followed by account number or… https://t.co/m0M9iVHhYE
number of retweets  0
number of likes  0
Stage 1 #Loadshedding to change to stage 2 from 14:00 till 22:00
number of retweets  36
number of li

The statue of Edward Colston has been unofficially replaced with a sculp…
number of retweets  2631
number of likes  0
RT @bendepear: WATCH @Channel4News NOW FOR THE NEWS !
number of retweets  2
number of likes  0
RT @Abukar_Arman: 1/2
Make no mistake: clanism is a loser's game. 
In their deadly quest to find surrogate claim to eternal superiority, cl…
number of retweets  13
number of likes  0
RT @MoradNews: BREAKING: Massive car bomb targets convoy carrying #Somalia army chief General Odawaa Yusuf Rage in #Mogadishu's Hodan distr…
number of retweets  14
number of likes  0
RT @jamesjonesfilm: The Rise of the Murdoch Dynasty starts tomorrow at 9pm on BBC Two.

https://t.co/3mvE44KDK5
number of retweets  7
number of likes  0
POPULARITY SCORE 2667
Reach score 93252
Getting data for iamsuede
#DefundThePolice https://t.co/RclTi752ar
number of retweets  0
number of likes  0
RT @thenodshow: Defund the police is a new, yet largely misunderstood positioning that noted abolitionist Bilphena Yahwo

@CyrilleNgantche @bueapikin @randzejude You come on here with fake names and fake accounts to try to pretend you ha… https://t.co/ubtLIXkKhr
number of retweets  0
number of likes  0
@CyrilleNgantche @bueapikin @randzejude Tell me what you do to change policy except for run your mouth on Twitter?… https://t.co/9WhKzQNjvN
number of retweets  0
number of likes  0
POPULARITY SCORE 2
Reach score 104262
Getting data for cx73
Calmness #Knysna https://t.co/HRFRlTmJK6
number of retweets  1
number of likes  14
End of Day #LakeBrenton https://t.co/hY7LhnmGth
number of retweets  1
number of likes  11
Beautiful morning in #J-Bay 8am &amp; 24degrees (berg wind) in the middle of winter 😃 #KabeljauwsRiver https://t.co/icwPsfSrVI
number of retweets  0
number of likes  9
Coffee drive #Sunrise #EasternCape https://t.co/HChr80QR4f
number of retweets  2
number of likes  19
@adamrady18 Yes we're in the middle of winter. Morning temp 10 degrees going up to 20 degrees 😎
number of retweets  0
number of likes  

The total number of COVID-19 cases in Gauteng is 107 070, with… https://t.co/IgKpPFvrku
number of retweets  9
number of likes  15
RT @DrZweliMkhize: As of today, the total number of confirmed #COVID19 cases is 298 292, the total number of deaths is 4 346 and the total…
number of retweets  507
number of likes  0
RT @DrZweliMkhize: #COVID19 Statistics in SA as at 14 July 2020. https://t.co/X8HmS3RC9i
number of retweets  522
number of likes  0
#COVID19 | ANYONE CAN GET COVID-19, REGARDLESS OF AGE, RACE OR GENDER. #PlayYourPart #StaySafe https://t.co/cCTIW3oSjY
number of retweets  12
number of likes  16
RT @bandilemasuku: We visited Dr George Mukhari Academic Hospital with Minister @DrZweliMkhize to check on some of the work done in the hos…
number of retweets  17
number of likes  0
POPULARITY SCORE 1098
Reach score 213607
Getting data for Aynte
RT @AhmedSolHoA: Our new @AfricaProg research paper on US 🇺🇸 policy in #Somalia 🇸🇴 by @PDWilliamsGWU is a must read!

Paper: https://t.co/f…
numbe

number of likes  0
RT @Diop_WB: Looking for bright PhD candidates or graduates from #Africa to join our #WBGFellowship program. Don’t miss out on this opportu…
number of retweets  105
number of likes  0
RT @kenyapics: Crystal clear night sky over Laikipia. Photo by Shaun Mousley https://t.co/ayOgK4YWyk
number of retweets  124
number of likes  0
POPULARITY SCORE 358
Reach score 49300
Getting data for willintune
Has any country in the world scratched its head quite as much as the U.K. over the mask wearing issue?
number of retweets  0
number of likes  5
Plus overnight curfew again.
number of retweets  0
number of likes  3
Alcohol ban has been reintroduced in South Africa. 
President Cyril Ramaphosa says between 40,000 and 50,000 deaths… https://t.co/7OReFYHW2z
number of retweets  40
number of likes  68
RT @sudanreeves: There could hardly be clearer evidence that real power in Sudan remains with the military—and the #RSF of Hemeti. These mu…
number of retweets  15
number of likes  0
Briti

Getting data for The_New_Age
Getting data for geoffreyyork
Nearly 11,000 excess deaths in South Africa from early May to early July, according to latest estimate released tod… https://t.co/NhlMPNl8Sy
number of retweets  12
number of likes  7
RT @pierredevos: Health department statistics last week put the Covid-19 death toll at 3600, but the estimated number of excess deaths attr…
number of retweets  19
number of likes  0
RT @AndrewWJHarding: This is important. Journalists are competitive, but we are also collegiate. We build on each others' reporting. And we…
number of retweets  2
number of likes  0
RT @Pumza_Fihlani: A health system on its knees : An eye-opening report on what’s really going on the Eastern Cape. Definitely worth a read…
number of retweets  6
number of likes  0
@khayadlanga @Mzo_honcho Here is last week's report: https://t.co/szNVjLJoOt
An update is to be released later today.
number of retweets  0
number of likes  1
POPULARITY SCORE 47
Reach score 274685
Getting data 

number of retweets  6
number of likes  3
Plenary National Assembly, 15 July 2020 https://t.co/E0oTvw6iTq
number of retweets  1
number of likes  1
Portfolio Committee on Defence and Military Veterans, 15 July 2020 https://t.co/xY8LT1aPwq
number of retweets  1
number of likes  1
Portfolio Committee on Social Development (NA) 15 July 2020 https://t.co/e3OGt2hH9C
number of retweets  1
number of likes  2
Portfolio Committee on Social Development (NA) 15 July 2020 https://t.co/QjDTnWJrK4
number of retweets  1
number of likes  2
POPULARITY SCORE 19
Reach score 670014
Getting data for SandtonCity
The Lumine features a bejewelled eggshell white dial with glistening @swarovski crystals. This exclusive design giv… https://t.co/KAwDeEFHMo
number of retweets  0
number of likes  0
Save big this winter with @Verimarksa winter demo clearance sale in #IconicSandton.
Get it while it's hot, offer va… https://t.co/XNFjQ5w5vV
number of retweets  0
number of likes  2
#BuildTomorrowTogether with @CheckersSA!

In [53]:
for r in df_gov:
    split_data = r.split('— ',maxsplit=1)[1].rsplit('(',maxsplit=1)
    name = split_data[0].split(',')[0].strip()
    handle =  split_data[1].rsplit(')',maxsplit=1)[0]
    print(f'{name} : {handle}')


AttributeError: 'int' object has no attribute 'split'